In [1]:
import sqlite3
import pandas as pd
import json
from glob import glob

In [36]:
# person = 'Katies'
person = 'Bens'
files = glob(f"C:/Users/jphav\Documents/School/4118/project/CSCI4118-Project/{person}Data/*.json")
# files = glob("C:/Users/jphav\Documents/School/4118/project/CSCI4118-Project/BensData/*.json")
totalUniqueSongs = []
for file in files:
    print(file)
    with open(f'{file}', encoding="utf-8") as json_data:
        data = json.load(json_data)
        if 'Library' in file:
            spotifyDf = pd.DataFrame(data['tracks'])
            
        else:
            spotifyDf = pd.DataFrame(data)
    # only keep the needed columns for sorting
    
    if 'Library' in file:
        keys = ['track', 'artist']
        
    elif 'Streaming' in file:
        keys = ['trackName', 'artistName']
    else:
        keys = ['master_metadata_track_name', 'master_metadata_album_artist_name']
    sortingDf = spotifyDf[keys]
    print("length of sorting df before dropping")
    print(len(sortingDf))
    sortingDf = sortingDf.drop_duplicates()
    print("length of sorting df after dropping")
    print(len(sortingDf))

    escapedDf = sortingDf.replace("'", "''", regex=True)

    connection = sqlite3.connect('track_metadata.db')
    # for every song in library make a sql query and retrieve the track id
    escapedDf['id'] = escapedDf.apply(lambda x: pd.read_sql_query(f"SELECT track_id FROM songs WHERE title='{x[0]}' AND artist_name='{x[1]}';", connection), axis=1)
    connection.close()
    validId =  [i.iloc[[0]]['track_id'] for i in escapedDf['id'] if i.empty == False]
    print(f"Adding {len(validId)} songs to the unique list")
    totalUniqueSongs = totalUniqueSongs + validId
    

C:/Users/jphav\Documents/School/4118/project/CSCI4118-Project/BensData\endsong_0.json
length of sorting df before dropping
16522
length of sorting df after dropping
2586
Adding 157 songs to the unique list
C:/Users/jphav\Documents/School/4118/project/CSCI4118-Project/BensData\endsong_1.json
length of sorting df before dropping
16526
length of sorting df after dropping
2614
Adding 161 songs to the unique list
C:/Users/jphav\Documents/School/4118/project/CSCI4118-Project/BensData\endsong_10.json
length of sorting df before dropping
16503
length of sorting df after dropping
2557
Adding 163 songs to the unique list
C:/Users/jphav\Documents/School/4118/project/CSCI4118-Project/BensData\endsong_11.json
length of sorting df before dropping
16500
length of sorting df after dropping
2578
Adding 157 songs to the unique list
C:/Users/jphav\Documents/School/4118/project/CSCI4118-Project/BensData\endsong_12.json
length of sorting df before dropping
16506
length of sorting df after dropping
2603
Add

In [38]:

validDf = pd.DataFrame(totalUniqueSongs)
validDf.drop_duplicates()
print(f"final number of unique songs {len(validDf)}")
validDf.to_csv(f"ValidSongs/{person}_ValidSongs.csv", index=False)

final number of unique songs 4845
